## Install the package dependencies before running this notebook

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy as np
import pickle5 as pickle
from glob import glob

"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

## Create a Torch.Dataset class for the training dataset

In [2]:

ROOT_PATH = "./"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]
def get_city_trajectories(city="palo-alto", split="train", normalized=False):

    
    outputs = None
    
    if split=="train":
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)[:int(n * 0.8)]
        
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)[:int(n * 0.8)]
        
    elif split == 'val':
        f_in = ROOT_PATH + 'train' + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)[int(n * 0.8):]
        
        f_out = ROOT_PATH + 'train' + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)[int(n * 0.8):]
    
    else:
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)

    return inputs, outputs
def get_cities_trajectories(split="train"):
    inputs_cities = None
    outputs_cities = None
    for city in cities:
        if split=="train":
            f_in = ROOT_PATH + split + "/" + city + "_inputs"
            inputs = pickle.load(open(f_in, "rb"))
            n = len(inputs)
            inputs = np.asarray(inputs)[:int(n * 0.8)]
            f_out = ROOT_PATH + split + "/" + city + "_outputs"
            outputs = pickle.load(open(f_out, "rb"))
            outputs = np.asarray(outputs)[:int(n * 0.8)]
        elif split == 'val':
            f_in = ROOT_PATH + 'train' + "/" + city + "_inputs"
            inputs = pickle.load(open(f_in, "rb"))
            n = len(inputs)
            inputs = np.asarray(inputs)[int(n * 0.8):]

            f_out = ROOT_PATH + 'train' + "/" + city + "_outputs"
            outputs = pickle.load(open(f_out, "rb"))
            outputs = np.asarray(outputs)[int(n * 0.8):] 
        else:
            f_in = ROOT_PATH + split + "/" + city + "_inputs"
            inputs = pickle.load(open(f_in, "rb"))
            n = len(inputs)
            inputs = np.asarray(inputs)
            outputs= None
        if inputs_cities is None:
            inputs_cities =inputs
        else:
            i = (np.concatenate((inputs_cities, inputs), axis=0))
            inputs_cities = i
        if outputs_cities is None:
            outputs_cities = outputs
        else:
            o = np.concatenate((outputs_cities, outputs), axis=0)
            outputs_cities= o
    return inputs_cities, outputs_cities  


class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_cities_trajectories(split=split)
        global_mean = np.mean(self.inputs, axis = (0,1), keepdims = True)
        global_std = np.std(np.sqrt(self.inputs[:, :, 0]**2 + self.inputs[:, :, 0]**2))
        print(global_mean)
        print(global_std)
        #if split == "train":
            #self.inputs = (self.inputs - global_mean)/global_std
            
            #self.outputs = (self.outputs - global_mean)/global_std
        #else:
            #self.inputs = (self.inputs - global_mean)/global_std
            #self.outputs = (self.outputs - global_mean)/global_std
        
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'miami' 
split = 'train'
train_dataset  = ArgoverseDataset(split = split)

[[[2791.70427041 1064.76260345]]]
4100.326502228328


In [3]:
val_dataset = ArgoverseDataset(split = 'val')
print(len(train_dataset))
print(len(val_dataset))

[[[2779.01081344 1063.5660105 ]]]
4124.162302888706
163051
40765


In [4]:
print(len(val_dataset))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

40765
cuda:0


## Create a DataLoader class for training

batch_sz = 16  # batch size 
train_loader = DataLoader(train_dataset,batch_size=batch_sz)

In [8]:
batch_sz = 128# batch size 
train_loader = DataLoader(train_dataset,batch_size=batch_sz)

In [9]:
from torch import nn, optim

class mlp(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.mlp = nn.Sequential(
            nn.Linear(100, 110),
            nn.LeakyReLU(),
            nn.Linear(110, 110),
            nn.LeakyReLU(),
            nn.Linear(110, 110),
            nn.LeakyReLU(),
            nn.Linear(110, 110),
            nn.LeakyReLU(),
            nn.Linear(110, 120)
        )
        
    def forward(self, x):
        x = x.reshape(-1, 100).float()
        x = self.mlp(x)
        x = x.reshape(-1, 60, 2)
        return x


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pred = mlp()
pred.to(device)
#opt = optim.LBFGS( pred.parameters(),lr=.005)
opt = optim.Adam(pred.parameters(), lr=.00001,weight_decay=0.9)
loss_func = nn.MSELoss()

In [21]:

for epoch in range(1000):
    
    total_loss = 0
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        inp = inp.to(device)
        out = out.to(device).float()
        preds = pred(inp)
        opt.zero_grad()
        loss = loss_func(preds,out)
        loss.backward()
        
        opt.step()
        
        total_loss += loss.item()
        
    print('epoch {} loss: {}'.format(epoch, total_loss))

epoch 0 loss: 214373.14879608154
epoch 1 loss: 215269.27643585205


KeyboardInterrupt: 

In [22]:

val_loader = DataLoader(val_dataset,batch_size=batch_sz)
inputs, outputs = get_cities_trajectories(split="val")
val_loss = 0
global_mean = np.mean(inputs, axis = (0,1), keepdims = True)
global_std = np.std(np.sqrt(inputs[:, :, 0]**2 + inputs[:, :, 0]**2))
gm = torch.from_numpy(global_mean).to(device)
gs = (global_std)
for i_batch, sample_batch in enumerate(val_loader):
    inp, out = sample_batch
    inp, out = inp.to(device), out.to(device)
   # print(out.size())
    preds = pred(inp)
    #preds = (preds * gs ) + gm
    loss = loss_func(preds,out)
    val_loss += loss.item()
print('loss: {}'.format(val_loss))

loss: 53968.561264396434


## Sample a batch of data and visualize 

In [24]:
import matplotlib.pyplot as plt
import random


def show_sample_batch(sample_batch):
    """visualize the trajectory for a batch of samples"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i,:,0], inp[i,:,1])
        axs[i].scatter(out[i,:,0], out[i,:,1])

        
for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    print(inp.shape, out.shape)
    break
    """
    TODO:
      implement your Deep learning model
      implement training routine
    """
    show_sample_batch(sample_batch)
    break
def get_traj_pred(test_traj_in):
    test_data = torch.tensor(test_traj_in).to(device)
    test_preds = pred(test_data.float())
    #est_preds.detach().numpy()
    return test_preds.cpu().detach().numpy()

torch.Size([128, 50, 2]) torch.Size([128, 60, 2])


In [25]:

print(pred)

mlp(
  (mlp): Sequential(
    (0): Linear(in_features=100, out_features=110, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=110, out_features=110, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=110, out_features=110, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=110, out_features=110, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=110, out_features=120, bias=True)
  )
)


In [28]:
num_pred_steps=60
all_preds = np.zeros(shape=(0, num_pred_steps * 2))
city_col = np.array([])

for city_name in cities:

    print("Processing city", city_name)
    
    test_traj_in, test_traj_out = get_city_trajectories(city=city_name, split="test")
    print(test_traj_in.shape)
    
    test_pred_arr = get_traj_pred(test_traj_in)
    print(test_pred_arr.shape)
    
    test_pred_arr_reshaped = np.reshape(test_pred_arr, newshape=(test_traj_in.shape[0], num_pred_steps * 2))
    print(test_pred_arr_reshaped.shape)

    all_preds = np.r_[all_preds, test_pred_arr_reshaped]
    city_col = np.r_[city_col, [str(i) + "_" + city_name for i in range(test_pred_arr.shape[0])]]

Processing city austin
(6325, 50, 2)
(6325, 60, 2)
(6325, 120)
Processing city miami
(7971, 50, 2)
(7971, 60, 2)
(7971, 120)
Processing city pittsburgh
(6361, 50, 2)
(6361, 60, 2)
(6361, 120)
Processing city dearborn
(3671, 50, 2)
(3671, 60, 2)
(3671, 120)
Processing city washington-dc
(3829, 50, 2)
(3829, 60, 2)
(3829, 120)
Processing city palo-alto
(1686, 50, 2)
(1686, 60, 2)
(1686, 120)


In [31]:
import pandas as pd
sub_df = pd.DataFrame(np.c_[city_col, all_preds], columns=[np.r_[["ID"], ["v" + str(i) for i in range(120)]]])
sub_df.to_csv('mlpcopy1.csv', index=None)

In [32]:
sample_sub = pd.read_csv('mlpcopy1.csv')
sample_sub.tail()

,ID,v0,v1,v2,v3,v4,v5,v6,v7,v8,...,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119
29838,1681_palo-alto,-1400.001709,-490.278046,-1398.489258,-491.101532,-1397.152588,-489.412506,-1396.301880,-490.349731,-1399.720947,...,-1398.270752,-490.230621,-1398.531982,-491.306915,-1399.746948,-490.097809,-1399.775024,-490.995911,-1398.764648,-491.166260
29839,1682_palo-alto,128.807724,-5.158374,127.608452,-2.295990,128.302399,-2.850490,128.636063,-3.031444,128.958725,...,127.355484,-1.271336,127.764763,-1.466748,128.860306,-0.626090,131.073715,-2.348398,128.352905,0.057585
29840,1683_palo-alto,-1438.665161,2158.782471,-1445.811035,2161.149902,-1448.554565,2158.391113,-1436.380615,2167.306641,-1438.968384,...,-1432.543335,2170.887207,-1438.330811,2158.953125,-1444.100342,2166.932373,-1435.093750,2157.525635,-1433.885620,2166.735596
29841,1684_palo-alto,1089.548218,1407.595215,1080.628540,1409.224854,1081.272827,1405.531982,1086.013184,1409.551636,1086.501465,...,1087.648438,1411.053711,1087.493042,1415.951172,1087.525635,1408.401855,1088.188721,1407.814209,1087.617188,1406.920044
29842,1685_palo-alto,5.193674,396.748840,9.440820,398.405975,4.385119,394.863464,5.193655,396.035736,8.839888,...,5.477061,395.916534,10.489322,397.052155,6.081702,394.673859,5.172849,397.361511,5.744670,390.975800
